In [140]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from functions.Random_forest import *
from sklearn.model_selection import GridSearchCV

from functions.Pre_processing_Casos_Graves import *
from functions.Random_forest import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection._search import GridSearchCV
from sklearn.model_selection._split import StratifiedKFold
from sklearn import svm

from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif, SelectFdr

import optuna
from sklearn.metrics import f1_score, precision_score, recall_score

from sklearn.model_selection import RepeatedKFold

### Lendo o Dataset:

In [141]:
df = pd.read_csv(r'C:\Users\mcacs\Desktop\Estudo_dados_de_covid\Data\Casos Graves\df_casos_graves')

### Utilizando a class 'random forest' para fazer o pré-processamento:

In [142]:
list_features = ["SYMPTOM_COUGH",'SYMPTOM_COLD','SYMPTOM_AIR_INSUFFICIENCY','SYMPTOM_FEVER','SYMPTOM_LOW_OXYGEN_SATURATION','SYMPTOM_BREATHING_CONDITION','SYMPTOM_TORACIC_APERTURE','SYMPTOM_THROAT_CONDITION',
                'SYMPTOM_HEADACHE','SYMPTOM_BODY_PAIN','SYMPTOM_DIARRHEA','SYMPTOM_RUNNY_NOSE','SYMPTOM_NOSE_CONGESTION','SYMPTOM_WEAKNESS','SYMPTOM_ANOSMIA_OR_HYPOSMIA','SYMPTOM_NAUSEA','SYMPTOM_LACK_OF_APPETITE',
                'SYMPTOM_ABDOMINAL_PAIN','SYMPTOM_CONSCIOUSNESS_DEGRADATION','DISEASE_HEART_OR_VASCULAR','DISEASE_DIABETES','DISEASE_HYPERTENSION','DISEASE_RESPIRATORY','DISEASE_OBESITY','DISEASE_KIDNEY','DISEASE_IMMUNOSUPPRESSION',
                'DISEASE_TABAGIST','DISEASE_ETILISM','DISEASE_LIVER','DISEASE_NEUROLOGIC','death', 'idade', 'sexo_Feminino','sexo_Masculino']

In [143]:
random_forest = Random_forest(df)
random_forest.run_graves()

df = random_forest.df
df = df[list_features]
df

,SYMPTOM_COUGH,SYMPTOM_COLD,SYMPTOM_AIR_INSUFFICIENCY,SYMPTOM_FEVER,SYMPTOM_LOW_OXYGEN_SATURATION,SYMPTOM_BREATHING_CONDITION,SYMPTOM_TORACIC_APERTURE,SYMPTOM_THROAT_CONDITION,SYMPTOM_HEADACHE,SYMPTOM_BODY_PAIN,...,DISEASE_KIDNEY,DISEASE_IMMUNOSUPPRESSION,DISEASE_TABAGIST,DISEASE_ETILISM,DISEASE_LIVER,DISEASE_NEUROLOGIC,death,idade,sexo_Feminino,sexo_Masculino
0,1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0.460870,0,1
1,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.000000,1,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.747826,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.573913,0,1
4,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.591304,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36430,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.669565,0,1
36431,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.643478,0,1
36432,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.643478,0,1
36433,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.634783,1,0


### Criação de uma função para instância do modelo, utilizando repeated K-fold:

In [144]:
from sklearn.metrics import f1_score, make_scorer

def KNN_model (X_train, y_train):
    k_range = range(1, 31)
    param_grid = {"n_neighbors": k_range}
    scores = ['accuracy','f1','precision','recall','roc_auc']

    f1 = make_scorer(f1_score , average='macro')

    #Init classifier
    knn = KNeighborsClassifier()

    # prepare the cross-validation procedure
    cv = RepeatedKFold(n_splits = 5, n_repeats = 10, random_state = 1)

    # Init Grid Search
    grid_cv = GridSearchCV(knn, param_grid, n_jobs = -1, cv=5, scoring = scores, refit='accuracy')

    # Fit 
    grid_cv.fit(X_train, y_train)

    #mean_test_score = grid_cv.cv_results_['mean_test_score']
    #std_test_score = grid_cv.cv_results_['std_test_score']
    best_score = grid_cv.best_score_
    best_params = grid_cv.best_params_

   

    print('#'*40)
    #print(f"mean_test_score: {grid_cv.cv_results_['mean_test_score']}")
    #print(f"std_test_score: {grid_cv.cv_results_['std_test_score']}")
    print('')
    print(f'best_score: {grid_cv.best_score_}')
    print(f'best_params_: {grid_cv.best_params_}')
    print('#'*40)
    print('')

    #return mean_test_score, std_test_score, best_score, best_params
    return best_score, best_params

### Aplicação do modelo para diferentes 'random_states':

In [145]:
#utilizando diferentes random_states para mudar os conjuntos de treino e teste
list_random_states = [0,12,42,50,54,68,79,81,92,112]

#listas para armazenar os resultados
#mean_test_scores = []
#std_test_scores = []
best_scores = []
best_params = []
X = df[["SYMPTOM_COUGH", "SYMPTOM_COLD", "SYMPTOM_AIR_INSUFFICIENCY", "SYMPTOM_FEVER", "SYMPTOM_LOW_OXYGEN_SATURATION", 'SYMPTOM_BREATHING_CONDITION','SYMPTOM_TORACIC_APERTURE','SYMPTOM_THROAT_CONDITION',
                'SYMPTOM_HEADACHE','SYMPTOM_BODY_PAIN','SYMPTOM_DIARRHEA','SYMPTOM_RUNNY_NOSE','SYMPTOM_NOSE_CONGESTION','SYMPTOM_WEAKNESS','SYMPTOM_ANOSMIA_OR_HYPOSMIA']]

y = df['death'].astype(int)

for random_state in list_random_states:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = random_state)

    #mean_test_score, std_test_score, best_score, best_param = KNN_model(X_train,y_train)
    best_score, best_param = KNN_model(X_train,y_train)

    #mean_test_scores.append(mean_test_score)
    #std_test_scores.append(std_test_score)
    best_scores.append(best_score)
    best_params.append(best_param)

########################################

best_score: 0.743687224256078
best_params_: {'n_neighbors': 28}
########################################

########################################

best_score: 0.742383798756263
best_params_: {'n_neighbors': 18}
########################################

########################################

best_score: 0.7419721584895067
best_params_: {'n_neighbors': 18}
########################################

########################################

best_score: 0.7444764466541722
best_params_: {'n_neighbors': 28}
########################################

########################################

best_score: 0.7458142834064139
best_params_: {'n_neighbors': 28}
########################################

########################################

best_score: 0.7439618786648763
best_params_: {'n_neighbors': 30}
########################################

########################################

best_score: 0.7445794803118141
best_params_: {'n_neighbors': 30}
#

In [146]:
X.dtypes

SYMPTOM_COUGH                    int64
SYMPTOM_COLD                     int64
SYMPTOM_AIR_INSUFFICIENCY        int64
SYMPTOM_FEVER                    int64
SYMPTOM_LOW_OXYGEN_SATURATION    int64
SYMPTOM_BREATHING_CONDITION      int64
SYMPTOM_TORACIC_APERTURE         int64
SYMPTOM_THROAT_CONDITION         int64
SYMPTOM_HEADACHE                 int64
SYMPTOM_BODY_PAIN                int64
SYMPTOM_DIARRHEA                 int64
SYMPTOM_RUNNY_NOSE               int64
SYMPTOM_NOSE_CONGESTION          int64
SYMPTOM_WEAKNESS                 int64
SYMPTOM_ANOSMIA_OR_HYPOSMIA      int64
dtype: object